After having trained a model, you can use the output.yml and an input sequence file to make predictions

In [1]:
from biotrainer.utilities import read_config_file
from biotrainer.inference import Inferencer

/Users/chdallago/git/biotrainer/venv/lib/python3.6/site-packages/torch/package/_directory_reader.py:17: UserWarning: Failed to initialize NumPy: numpy.core.multiarray failed to import (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:68.)
  _dtype_to_storage = {data_type(0).dtype: data_type for data_type in _storages}


ImportError: cannot import name 'nullcontext'

In [2]:
out_config_path = '../examples/residue_to_class/output/out.yml'
out_config = read_config_file(out_config_path)

Let's find out how well the model performs on the test set.

In [3]:
print(f"For the {out_config['model_choice']}, the metrics on the test set are:")
for metric in out_config['test_iterations_results']['metrics']:
    print(f"\t{metric} : {out_config['test_iterations_results']['metrics'][metric]}")

For the CNN, the metrics on the test set are:
	accuracy : 4.0
	loss : 2.524291515350342


First we need to create the embeddings for the sequences we are interested in

In [4]:
from bio_embeddings.embed import ProtTransT5XLU50Embedder

In [5]:
embedder = ProtTransT5XLU50Embedder()

In [11]:
sequences = [
    "PROVTEIN",
    "SEQVENCESEQVENCE"
]

In [12]:
embeddings = embedder.embed_many(sequences)

Next we generate an inference object from the out config of our training run

In [13]:
inferencer = Inferencer(**out_config)

In [14]:
predictions = inferencer.from_embeddings(embeddings)

We can inspect the predictions

In [15]:
for sequence, prediction in zip(sequences, predictions):
    print(sequence)
    print(prediction)

PROVTEIN
DVCDDCCDVDDDDDDD
SEQVENCESEQVENCE
DVCDVVDDDVCDVVDD
